In [1]:
import pandas as pd
fp = "../data/incident_event_log.csv"
df = pd.read_csv(fp)
df.head()

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
0,INC0000045,New,True,0,0,0,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
1,INC0000045,Resolved,True,0,0,2,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
2,INC0000045,Resolved,True,0,0,3,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
3,INC0000045,Closed,False,0,0,4,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
4,INC0000047,New,True,0,0,0,True,Caller 2403,Opened by 397,29/2/2016 04:40,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 81,1/3/2016 09:52,6/3/2016 10:00


In [7]:
support_cols =["assignment_group", "assigned_to"]
df_sg = df[support_cols]
df_sg = df_sg.replace(to_replace = '?', value = 'UNKNOWN')
df_sg = df_sg.astype("category")

In [8]:
df_sg.head()

,assignment_group,assigned_to
0,Group 56,UNKNOWN
1,Group 56,UNKNOWN
2,Group 56,UNKNOWN
3,Group 56,UNKNOWN
4,Group 70,Resolver 89


In [9]:
cols = df_sg.columns.tolist()
vocab_size = 0
for c in cols:
    print("Num unique vals for category " + str(c) + " = " + str(df_sg[c].nunique()))
    vocab_size += df_sg[c].nunique()
print("Vocab size: %s" % vocab_size)

Num unique vals for category assignment_group = 79
Num unique vals for category assigned_to = 235
Vocab size: 314


In [3]:
numeric = ['sys_mod_count', 'reopen_count']
dfn = df[numeric]
dcols = []
for col in numeric:
    dlabel = 'D_' + col
    labels = [dlabel +'_' + str(c) for c in range(5)]
    dcols.append(dlabel)
    dfn[dlabel] = pd.qcut(dfn[col].rank(method='first'),5, labels = labels, duplicates = 'drop')
    
attributes = df.columns.tolist()
remove = [ 'made_sla', 'opened_at', 'resolved_at','sys_created_at', 'caller_id', 'closed_at',\
          'notify', 'sys_updated_by','sys_created_by', 'number', 'sys_updated_at', 'reassigned', 'reassignment_count' ]
exclude = remove + numeric
keep = list(set(attributes) - set(exclude)) 
keep

/home/admin2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


['incident_state',
 'closed_code',
 'contact_type',
 'location',
 'knowledge',
 'cmdb_ci',
 'category',
 'active',
 'vendor',
 'opened_by',
 'assigned_to',
 'urgency',
 'subcategory',
 'priority',
 'resolved_by',
 'rfc',
 'assignment_group',
 'problem_id',
 'impact',
 'caused_by',
 'u_priority_confirmation',
 'u_symptom']

In [4]:
df_cat_vars = df[keep]
df_cat_vars = df_cat_vars.replace(to_replace = '?', value = 'UNKNOWN')
df_cat_vars =  pd.concat([df_cat_vars, dfn[dcols]], axis = 1)
df['made_sla'] = df['made_sla'].map({True: 1, False: 0})

df = df.reset_index()

In [5]:
cols = df_cat_vars.columns.tolist()
vocab_size = 0
for c in cols:
    print("Num unique vals for category " + str(c) + " = " + str(df_cat_vars[c].nunique()))
    vocab_size += df_cat_vars[c].nunique()
print("Vocab size: %s" % vocab_size)

Num unique vals for category incident_state = 9
Num unique vals for category closed_code = 18
Num unique vals for category contact_type = 5
Num unique vals for category location = 225
Num unique vals for category knowledge = 2
Num unique vals for category cmdb_ci = 51
Num unique vals for category category = 59
Num unique vals for category active = 2
Num unique vals for category vendor = 5
Num unique vals for category opened_by = 208
Num unique vals for category assigned_to = 235
Num unique vals for category urgency = 3
Num unique vals for category subcategory = 255
Num unique vals for category priority = 4
Num unique vals for category resolved_by = 217
Num unique vals for category rfc = 182
Num unique vals for category assignment_group = 79
Num unique vals for category problem_id = 253
Num unique vals for category impact = 3
Num unique vals for category caused_by = 4
Num unique vals for category u_priority_confirmation = 2
Num unique vals for category u_symptom = 526
Num unique vals fo

In [6]:
df_cat_vars.dtypes

incident_state               object
closed_code                  object
contact_type                 object
location                     object
knowledge                      bool
cmdb_ci                      object
category                     object
active                         bool
vendor                       object
opened_by                    object
assigned_to                  object
urgency                      object
subcategory                  object
priority                     object
resolved_by                  object
rfc                          object
assignment_group             object
problem_id                   object
impact                       object
caused_by                    object
u_priority_confirmation        bool
u_symptom                    object
D_sys_mod_count            category
D_reopen_count             category
dtype: object

In [7]:
df_cat_vars = df_cat_vars.astype("category")

In [8]:
df_cat_vars.dtypes

incident_state             category
closed_code                category
contact_type               category
location                   category
knowledge                  category
cmdb_ci                    category
category                   category
active                     category
vendor                     category
opened_by                  category
assigned_to                category
urgency                    category
subcategory                category
priority                   category
resolved_by                category
rfc                        category
assignment_group           category
problem_id                 category
impact                     category
caused_by                  category
u_priority_confirmation    category
u_symptom                  category
D_sys_mod_count            category
D_reopen_count             category
dtype: object

In [9]:
pd.get_dummies(data=df_cat_vars)

,incident_state_-100,incident_state_Active,incident_state_Awaiting Evidence,incident_state_Awaiting Problem,incident_state_Awaiting User Info,incident_state_Awaiting Vendor,incident_state_Closed,incident_state_New,incident_state_Resolved,closed_code_UNKNOWN,...,D_sys_mod_count_D_sys_mod_count_0,D_sys_mod_count_D_sys_mod_count_1,D_sys_mod_count_D_sys_mod_count_2,D_sys_mod_count_D_sys_mod_count_3,D_sys_mod_count_D_sys_mod_count_4,D_reopen_count_D_reopen_count_0,D_reopen_count_D_reopen_count_1,D_reopen_count_D_reopen_count_2,D_reopen_count_D_reopen_count_3,D_reopen_count_D_reopen_count_4
0,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141707,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,1
141708,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
141709,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
141710,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
